In [9]:
import pandas as pd
import numpy as np
import ta
from sklearn.model_selection import train_test_split


In [2]:
def clean_ds (df):
    for i in range(1, 4):  
        df[f'X_t-{i}'] = df['Close'].shift(i)

    # Shift Close Column up by 5 rows
    df['Pt_5'] = df['Close'].shift(-5)

    #Agregamos RSI
    rsi_data = ta.momentum.RSIIndicator(close= df['Close'], window=28)
    df['RSI'] = rsi_data.rsi()

    # La Y
    df['Y_BUY'] = df['Close'] * (1 + 0.02) < df['Pt_5']
    df['Y_SELL'] = df['Close'] * (1 - 0.02) > df['Pt_5']
    
    df['Y_BUY'] = df['Y_BUY'].astype(int)
    df['Y_SELL'] = df['Y_SELL'].astype(int)

    return df

In [3]:
data_1 = pd.read_csv('../data/aapl_1h_train.csv')
data_1

,Date,Open,High,Low,Close,Adjusted_close,Volume
0,2000-01-03,104.8768,112.5040,101.6848,111.9328,0.8472,535796800
1,2000-01-04,108.2480,110.6224,101.1920,102.5024,0.7758,512377600
2,2000-01-05,103.7456,110.5664,102.9952,104.0032,0.7872,778321600
3,2000-01-06,106.1200,107.0048,94.9984,94.9984,0.7190,767972800
4,2000-01-07,96.4992,101.0016,95.5024,99.5008,0.7531,460734400
...,...,...,...,...,...,...,...
5279,2020-12-24,131.3200,133.4600,131.1000,131.9700,129.6797,54930100
5280,2020-12-28,133.9900,137.3400,133.5100,136.6900,134.3178,124486200
5281,2020-12-29,138.0500,138.7900,134.3400,134.8700,132.5294,121047300
5282,2020-12-30,135.5800,135.9900,133.4000,133.7200,131.3994,96452100


In [4]:
data_ejem = clean_ds(data_1)
data_ejem

,Date,Open,High,Low,Close,Adjusted_close,Volume,X_t-1,X_t-2,X_t-3,Pt_5,RSI,Y_BUY,Y_SELL
0,2000-01-03,104.8768,112.5040,101.6848,111.9328,0.8472,535796800,NaN,NaN,NaN,97.7536,NaN,0,1
1,2000-01-04,108.2480,110.6224,101.1920,102.5024,0.7758,512377600,111.9328,NaN,NaN,92.7472,NaN,0,1
2,2000-01-05,103.7456,110.5664,102.9952,104.0032,0.7872,778321600,102.5024,111.9328,NaN,87.1920,NaN,0,1
3,2000-01-06,106.1200,107.0048,94.9984,94.9984,0.7190,767972800,104.0032,102.5024,111.9328,96.7456,NaN,0,0
4,2000-01-07,96.4992,101.0016,95.5024,99.5008,0.7531,460734400,94.9984,104.0032,102.5024,100.4416,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5279,2020-12-24,131.3200,133.4600,131.1000,131.9700,129.6797,54930100,130.9600,131.8800,128.2300,NaN,47.057776,0,0
5280,2020-12-28,133.9900,137.3400,133.5100,136.6900,134.3178,124486200,131.9700,130.9600,131.8800,NaN,50.210963,0,0
5281,2020-12-29,138.0500,138.7900,134.3400,134.8700,132.5294,121047300,136.6900,131.9700,130.9600,NaN,49.042950,0,0
5282,2020-12-30,135.5800,135.9900,133.4000,133.7200,131.3994,96452100,134.8700,136.6900,131.9700,NaN,48.306613,0,0


In [7]:
close_data = data_ejem[['Close', 'X_t-1', 'X_t-2', 'X_t-3', 'RSI', 'Y_BUY']]
close_data


,Close,X_t-1,X_t-2,X_t-3,RSI,Y_BUY
0,111.9328,NaN,NaN,NaN,NaN,0
1,102.5024,111.9328,NaN,NaN,NaN,0
2,104.0032,102.5024,111.9328,NaN,NaN,0
3,94.9984,104.0032,102.5024,111.9328,NaN,0
4,99.5008,94.9984,104.0032,102.5024,NaN,0
...,...,...,...,...,...,...
5279,131.9700,130.9600,131.8800,128.2300,47.057776,0
5280,136.6900,131.9700,130.9600,131.8800,50.210963,0
5281,134.8700,136.6900,131.9700,130.9600,49.042950,0
5282,133.7200,134.8700,136.6900,131.9700,48.306613,0


In [8]:
close_data['Y_BUY'].value_counts()


Y_BUY
0    3318
1    1966
Name: count, dtype: int64

In [ ]:
x_train, y_train, x_test, y_test = train_test_split(close_data[['Close', 'X_t-1', 'X_t-2', 'X_t-3', 'RSI']], close_data['Y_BUY'], test_size=0.2, random_state=42)

In [20]:
from machine_learning.xgb_def import XGBOptimizer


ModuleNotFoundError: No module named 'machine_learning'

In [ ]:
xgb_optuna(x_train, y_train, x_test, y_test)